<div class="headline">
Language Technology / Sprachtechnologie
<br><br>
Wintersemester 2019/2020
</div>
<br>
<div class="description">
    Übung zum Thema <i id="topic">"Grammatical Error Correction/Spelling Correction"</i>
    <br><br>
    Deadline Abgabe: <i #id="submission">Thursday, 16.01.2020 (23:55 Uhr)</i>
</div>

In [4]:
from IPython.core.display import HTML
HTML("<style>" + open("style.css").read() + "</style>")

In [ ]:
import numpy as np
from nltk import bigrams
from nltk import FreqDist
from nltk.tokenize import word_tokenize

## Warm-Up

<div class="task_description">
    <i class="task">Task 10.1:</i> <br>
</div>

Which statements are true?

1. Essay Scoring, Machine Translation and Grammatical Error correction can technically be treated as the same task.

2. The Levenshtein distance between two strings is the minimum number of edit operations (substitution, insertion, deletion) that is necessary to transform one string into the other.

3. Spelling correction: Longer words are easier to correct than shorter words.

4. The Levenshtein distance is based on the assumption that there is exactly one optimal alignment between two strings.

5. The Levenshtein distance can only be computed if the strings have the same length. If the strings have different lengths, we need to compute the Damerau-Levenshtein distance.

<strong style="color: blue">Lösung:</strong>

1. False. Machine Translation and Grammatical Error Correction can indeed be seen as variants of the same task, namely a sequence-to-sequence problem in which we want to transform an input sequence (of variable length) to an output sequence (which not necessarily has the same length). Essay Scoring, in contrast, is a classification task in which an input sequence is assigned a score.

2. True.

3. Often true (but not always). Shorter words tend to have more "neighbors", i.e. words that differ by only one letter (Levenshtein distance = 1), for example "poice" could be corrected to "police", "price", "voice", "poise"... with one edit, while long words often just have a unique correction candidate  (e.g. "abministration" : "administration"). The statement is not generally true, of course: How hard it is to correct a misspelling is  dependent on many other factors, e.g. the type of error, the correction model (different weights for different edits) etc.

4. False. There can be several alignments between two strings which require the same (minimum) number of edits.

5. False. Levenshtein distance can be computed between strings of any length. The Damerau-Levenshtein distance is a variant of the Levenshtein distance in which also the transposition of two characters counts as one edit operation.

## Edit Distance Manually

<div class="task_description">
    <i class="task">Task 10.2:</i><br>
</div>

<div class="task_description">
   <i class="subtask">10.2.1</i> <i class="l1">L1</i> <br>
</div>

Given the following alignment between the words "rot" and "Wort", how many and which edit operations are needed to convert one string into the other? (Do not count copying an identical character as an edit operation here)

| _ | r | o | t |
|---|---|---|---|
| W | o | r | t |



<strong style="color: blue">Lösung:</strong>

3 edit operations: insert "W", replace "r" with "o", replace "o" with "r"

<div class="task_description">
   <i class="subtask">10.2.2</i> <i class="l2">L2</i> <br>
</div>

Is this already the optimal alignment? Compute manually the minimum edit distance (Levenshtein distance) between the source word "rot" and the target word "Wort".


<strong style="color: blue">Lösung:</strong>

|   |   | W | o | r | t |
|---|---|---|---|---|---|
|   | 0 | 1 | 2 | 3 | 4 |
| r | 1 | 1 | 2 | 2 | 3 |
| o | 2 | 2 | 1 | 2 | 3 |
| t | 3 | 3 | 2 | 2 | 2 |

The minimum edit distance is 2.

<div class="task_description">
   <i class="subtask">10.2.3</i> <i class="l2">L2</i> <br>
</div>

Find the best alignment(s) between "rot" and "Wort".

<strong style="color: blue">Lösung:</strong>
    
| r | o | _ | t |
|---|---|---|---|
| W | o | r | t |

## Edit Distance Automatically


<div class="task_description">
    <i class="task">Task 10.3:</i><br>
</div>

<div class="task_description">
   <i class="subtask">10.3.1</i> <i class="l1">L1</i> <br>
</div>

What does the following code do?

In [ ]:
def levenshtein(seq1, seq2):
    
    size_x = len(seq1) + 1
    size_y = len(seq2) + 1
    
    matrix = np.zeros ((size_x, size_y))
    
    for x in range(size_x):
        matrix [x, 0] = x
    for y in range(size_y):
        matrix [0, y] = y

    for x in range(1, size_x):
        for y in range(1, size_y):
            
            if seq1[x-1] == seq2[y-1]:
                cost = 0
            else:
                cost = 1
                
            matrix [x,y] = min(
                matrix[x-1,y] + 1,
                matrix[x-1,y-1] + cost,
                matrix[x,y-1] + 1
                )
            
    print (matrix)
    return (matrix[size_x - 1, size_y - 1])

print(levenshtein("rot", "Wort"))

<strong style="color: blue">Lösung:</strong>
    
This is an implementation of Levenshtein distance (dynamic programming), which outputs the computed matrix and the final minimum edit distance.

<div class="task_description">
   <i class="subtask">10.3.2</i> <i class="l2">L2</i> <br>
</div>

Change the function above in a way so that wrong lettercase (e.g. changing "r" to "R" only has a cost of 0.5).

<strong style="color: blue">Lösung:</strong>

In [ ]:
def levenshtein(seq1, seq2):
    
    size_x = len(seq1) + 1
    size_y = len(seq2) + 1
    
    matrix = np.zeros ((size_x, size_y))
    
    for x in range(size_x):
        matrix [x, 0] = x
    for y in range(size_y):
        matrix [0, y] = y
    
        
    for x in range(1, size_x):
        for y in range(1, size_y):
            
            if seq1[x-1] == seq2[y-1]:
                cost = 0
            
            ### cost for only changing letter case is set to 0.5
            elif seq1[x-1].lower() == seq2[y-1].lower():
                cost = 0.5
            
            else:
                cost = 1
                
            matrix [x,y] = min(
                matrix[x-1,y] + 1,
                matrix[x-1,y-1] + cost,
                matrix[x,y-1] + 1
                )
            
    #print (matrix)
    return (matrix[size_x - 1, size_y - 1])

print(levenshtein("Rot", "rot"))

<div class="task_description">
   <i class="subtask">10.3.3</i> <i class="l1">L1</i> <br>
</div>

Think of other adjustments to the code that could be useful for using Levenshtein distance in a spellchecking algorithm.

<strong style="color: blue">Lösung:</strong>

Some ideas are:

- Multi-letter graphemes like "sch", "ch", "qu" in German should be treated like fixed units rather than single letters

- Depending on the application/target group it would for example be useful to reduce the costs for 
 
 - substituting a letter with another letter that is adjacent to it on a standard keyboard (e.g. "a" and "s")
 - substituting two letters with a similar pronunciation (e.g. "f" and "v")
 - insertion if the missing character is part of a double consonant (e.g. "komen" for "kommen")
 

## Real-Word Spelling Errors

<div class="task_description">
    <i class="task">Task 10.4:</i><br>
</div>

<div class="task_description">
   <i class="subtask">10.4.1</i> <i class="l1">L1</i> <br>
</div>

What does the following code do?

In [ ]:
with open("deu_news_2015_30K-sentences.txt", mode="r", encoding="utf-8") as f:
    corpus = f.read().splitlines()

sentences = [word_tokenize(sentence) for sentence in corpus]
sentences = [sentence[1:] for sentence in sentences]

bi = [bigrams(sentence) for sentence in sentences]
bi = [bigram for sublist in bi for bigram in sublist]

fdist = FreqDist(bi)
print(fdist.most_common(5))

<strong style="color: blue">Lösung:</strong>

A file is read in which contains one sentence per line. The file is tokenized and the first token is deleted (contains the line number). Each sentence is split into bigrams (so that there are no bigrams across sentence boundaries because the sentences do not form a coherent text). All bigrams are then stored in one list, over which a frequency distribution is calculated. The five most frequent bigrams are then printed.

<div class="task_description">
   <i class="subtask">10.4.2</i> <i class="l2">L2</i> <br>
</div>

Take the sentence "Das ist seid Jahren ein Problem". Which bigram(s) is/are the most infrequent ones given the news corpus provided above? What could it indicate?

<strong style="color: blue">Lösung:</strong>

In [ ]:
sent = "Das ist seid Jahren ein Problem."
sent = word_tokenize(sent)
sent_bi = bigrams(sent)

for bi in sent_bi:
    print(bi, fdist[bi])

The low frequency of the bigrams including "seid" suggest that this could be a real-word spelling error. 

<div class="task_description">
   <i class="subtask">10.4.3</i> <i class="l2">L2</i> <br>
</div>

How many different unigram **types** are there in the news corpus? Which are the five most frequent ones?

<strong style="color: blue">Lösung:</strong>

In [ ]:
unigrams = [word for sublist in sentences for word in sublist]
uni_types = set(unigrams)
print(len(uni_types))
uni_fdist = FreqDist(unigrams)
print(uni_fdist.most_common(5))

<div class="task_description">
   <i class="subtask">10.4.4</i> <i class="l2">L2</i> <br>
</div>

Which words from the news corpus have a Levenshtein distance of 1 to "seid"? These will be the correction candidates for our real-word error.

<strong style="color: blue">Lösung:</strong>

In [ ]:
candidates = []
for word in uni_types:
    if levenshtein(word, "seid") == 1:
        candidates.append(word)
print(candidates)

<div class="task_description">
   <i class="subtask">10.4.5</i> <i class="l3">L3</i> <br>
</div>

Which of the candidate words is most likely to appear in the bigrams ("ist", x) and (x, "Jahren")?

<strong style="color: blue">Lösung:</strong>

In [ ]:
ist_x = {word: fdist["ist", word] for word in candidates}
print(max(ist_x))

x_Jahren = {word: fdist[word, "Jahren"] for word in candidates}
print(max(x_Jahren))

# Homework

<div class="task_description">
    <i class="task">10.1.</i> :::10 Homework points:::
</div>


The text *spelling_homework.csv* was written by a primary school child. It contains 75 tokens, of which 30 are spelled incorrectly. There is one token per line. The first column contains the original spelling and the second column the gold solution, i.e. the correct spelling of every word. The text is about two kids, Lea and Lars, and their dog Dodo. 

Implement a spelling corrector that predicts the correct spelling for each word. This means, it has to decide for every word whether it needs correcting and if this is the case, it has to provide one correction (not multiple correction candidates -- you have to decide for one). A correction counts as correct if it exactly matches the gold correction of the word (including letter case). You get the full homework points if your spellchecker reduces the number of incorrectly spelled words from 30 to 27 or less (keep in mind that your spelling corrector does not know beforehand which of the words are spelled incorrectly, so it may happen that you change a correct word to something incorrect, which then counts as a misspelling as well). 

You have to use the provided template for your solution. The function ``correctText()`` takes as input a list of tokens and outputs a list of corrected tokens (the lists have to have the same length). The other functions should not be changed. They can be used to evaluate your corrections.  

Note: It is not a valid solution to simply map every word to its correction ;-) Your solution should be general enough to work for other texts as well (see Challenge below).


### Challenge

::: 1 extra exam bonus point :::

Your homework submission will automatically enter a challenge. Your spelling corrector will be tested on another text with similar spelling errors and 1 extra exam bonus point will be awarded to the 3 teams who achieve the highest number of correct words in the test text. The final results will be presented in the lecture.

**Template:**

In [ ]:
def correctText(listOfTokens):
    
    #the spell checking functionality goes here
    #right now the function just outputs every word as it is
    corrections = [word for word in listOfTokens]
    
    return corrections
    
    
def evaluate(corrected_tokens, original_spellings, solution):
    
    #make sure that all three lists have the same length
    assert len(corrected_tokens) == len(original_spellings) == len(solution), "Error. The lists with proposed corrections, original tokens and the solution do not have the same lengths."
    
    #count nuber of incorrect tokens (=whenever there is a difference between the proposed correction and the gold solution)
    incorrect_tokens = sum(1 for i in range(len(corrected_tokens)) if corrected_tokens[i] != solution[i])
    
    print("Number of incorrect tokens in the text:", incorrect_tokens)
    

def correct_and_evaluate(filename):
    
    #read in text
    with open(filename, mode="r", encoding="utf-8") as f:
        text = f.read().splitlines()
    text = [line.split("\t") for line in text]
    
    #comment in to see the text
    #for line in text:
    #    print(line[0], line[1])

    #extract original spelling and solution for every word
    original_spellings = [line[0] for line in text]
    solution = [line[1] for line in text]
    
    #correct tokens
    proposed_corrections = correctText(original_spellings)
    
    #evaluate corrections
    evaluate(proposed_corrections, original_spellings, solution)
    

In [ ]:
correct_and_evaluate("spelling_homework.csv")